#Very Important Spark Learning - BY LEARNING this PROGRAM - WE BECOME A DATA ENGINEER (DATA CURATION DEVELOPER & DATA ANALYST)
Only the usage of SQL/Declarative Language Programming
###Points to consider using SQL rather than DSL
1. We can't use pure sql alone for achieving all functionalities.
2. We will use SQL/DSL appropriately as and when it is applicable.
3. We use DSL in data ingestion/egress mostly, using SQL in ingestion/egress is bit challenging.
4. Performance wise usage of SQL or DSL is going to almost same, because any way SQL/DSL internally spark is optimizing the code and convert the code in RDD.
5. Anytime we can represent DF to SQL or SQL to DF easily.

In [0]:
%python
rawdf1=spark.read.csv("/Volumes/workspace/default/volume1/custs",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession").select("*")
#If i want to further write SQL queries rather than writing DSL, we can create a view on top of the dataframe
rawdf1.createOrReplaceTempView("rawdf1view")
spark.sql("select * from rawdf1view").show()
#display(rawdf1)

In [0]:
%sql
--pure sql without any FBP/DSL
SELECT * from rawdf1view limit 10;

In [0]:
%sql
--For extraction/load operations with multiple features, preferably use DSL and not SQL
--For initial dataset creation or for ingestion/egress DSL way of creating dataframe is better, because not all the options are supported in SQL ingestion
CREATE OR REPLACE TEMPORARY VIEW rawdf1view
(
  id INT,
  firstname STRING,
  lastname STRING,
  age INT,
  profession STRING
)
USING CSV
OPTIONS (
  path "/Volumes/workspace/default/volume1/custs",
  header "false",
  inferSchema "false" 
);

select * from rawdf1view limit 20;

For creating initial dataframe, SQL is not better means, if I wanted to write SQL query after dataframe creation? We have to represent Dataframe to table/view format, so we can write SQL query..

In [0]:
%python
#Fundamentally we need to know 3 things?
#1. How to represent DF to View?
#2. How to represent View to DF?
#3. How to represent View to View?

#1. How to represent DF to View?
#How to represent dataframe to tempview? So we can write sql rather than dsl for transformation and analytics.
rawdf1.select("*").where("age between 18 and 65").limit(10).show()#DSL way
#or
rawdf1.createOrReplaceTempView("rawdf1view")#now we are good to write ISO sql queries
spark.sql("select * from rawdf1view where age between 18 and 65 limit 10").show()#SQL way

#2. How to represent View to DF?
df2=spark.sql("select * from rawdf1view where age between 18 and 65 limit 10")
df2.show(10)

#3. How to represent View to View?
spark.sql("select * from rawdf1view where age between 18 and 65 limit 10").createOrReplaceTempView("view2")
display(spark.sql("select * from view2"))
#or
spark.sql("create or replace temp view view3 as select * from rawdf1view where age between 18 and 65 limit 10")
display(spark.sql("select * from view3"))

##**1. Data Munging** - (Cleanup) Process of transforming and mapping data from Raw form into Tidy(usable) format with the intent of making it more appropriate and valuable for a variety of downstream purposes such for further Transformation/Enrichment, Egress/Outbound, analytics, Datascience/AI application & Reporting

![Stage1](stage1.png)

**Passive Data Munging** - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns. <br>

**Active Data Munging**
1. Combining Data + Schema Evolution/Merging/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

###a. Passive Data Munging - 
- Visible - Data Discovery/Data Exploration/ EDA (Exploratory Data Analytics) (every layers ingestion/transformation/analytics/consumption) - Performing an (Data Exploration) exploratory data analysis of the raw data to identify the attributes and patterns.

####1. Visibily/Manually opening the file we found couple of data patterns (Manual Exploratory Data Analysis)
- It is a Structured data with comma seperator (CSV)
- No Header, No comments, footer is there in the data
- Total columns are (seperator + 1)
- Data Quality 
- - Null columns and null records are there
- - duplicate rows & Duplicate id keys
- - format issues are there (age is not in number format eg. 7-7)
- - Uniformity issues (Artist, artist)
- - Number of columns are more or less than the expected
- eg. 4000011,Francis,McNamara,47,Therapist,NewYork & 4000014,Beth,Woodard,65
- - Identification of data type

####2. Programatically lets try to find couple of data patterns applying EDA - passively (without modifying, just for description).


In [0]:
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1.show(20,False)
display(rawdf1.take(20))
display(rawdf1.sample(.1))

In [0]:
%sql
--We can directly create tempview from the source
CREATE OR REPLACE TEMPORARY VIEW rawdf1view
(
  id INT,
  firstname STRING,
  lastname STRING,
  age INT,
  profession STRING
)
USING CSV
OPTIONS (
  path "/Volumes/workspace/default/volume1/custs",
  header "false",
  inferSchema "false" 
);

select * from rawdf1view limit 20;

In [0]:
%sql
select * from rawdf1view WHERE RAND() <= 0.1;

In [0]:
#Important passive EDA structure functions we can use
rawdf1.printSchema()#I am realizing the id & age columns are having some non numerical values (supposed to be numeric)
print(rawdf1.columns)#I am understanding the column numbers/order and the column names
print(rawdf1.dtypes)#Realizing the datatype of every columns (even we can do programattic column & type identification for dynamic programming)
for i in rawdf1.dtypes:
    if i[1]=='string':
        print(i[0])

print(rawdf1.schema)#To identify the structure of the data in the StructType and StructField format

In [0]:
%sql
DESCRIBE rawdf1view;

In [0]:
#Important passive EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - 
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
rawdf1.createOrReplaceTempView("rawdf1view")
print("actual count of the data","select count(*) from rawdf1view")
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given cid column count",rawdf1.dropDuplicates(['id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

In [0]:
%sql
--inline view (from clause subquery)
SELECT COUNT(*) AS dedup_all_columns_count
FROM (SELECT DISTINCT * FROM rawdf1view) t;

In [0]:
%sql
--Importantly, we are using qualify function (advanced feature) to avoid additional temporary views (refer next cell)
--df.dropDuplicates(['id']).count()
SELECT COUNT(*) AS dedup_id_count
FROM (
  SELECT *
  FROM rawdf1view
  qualify ROW_NUMBER() OVER (PARTITION BY id ORDER BY id)=1) inlineview;

In [0]:
%sql
SELECT COUNT(*) AS dedup_id_count
FROM 
(
  select id,age from 
(
  SELECT *,ROW_NUMBER() OVER (PARTITION BY id ORDER BY id) rno
  FROM rawdf1view) inlineview
  where rno = 1) inlineview2;
  

###**b. Active Data Munging**
1. Combining Data + Schema Evolution/Merging (Structuring)
2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)
3. De Duplication and Levels of Standardization () of Data to make it in a usable format (Dataengineers/consumers)

In [0]:
from pyspark.sql.session import SparkSession#15lakhs
spark=SparkSession.builder.appName("WD36 - ETL Pipeline - Bread & Butter").getOrCreate()#3 lakhs LOC by Databricks (for eg. display, delta, xml)

####1. **Structuring** - Combining Data + Schema Evolution/Merging 

When do we go for Schema Evolution?<br>
Over the time, if no. of col are keep added by source<br>
Serialization  while writing+ mergeSchema while reading<br>
When do we go for Schema Merging?<br>
In a given day, If we get multiple files of related (not same) structure<br>
After reading in dataframe format -> unionByName + allowMissingColumns

In [0]:
#Extraction (Ingestion) methodologies
#1. Single file
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified")
#2. Multiple files (with same or different names)
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified","/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified"])
#3. Multiple files in multiple paths or sub paths
rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/","/Volumes/workspace/wd36schema/ingestion_volume/staging/"],recursiveFileLookup=True,pathGlobFilter="custsm*")


In [0]:
#Active Data munging...
#When you go for Schema Merging/Melting and Schema Evolution?
#Schema Merging/Melting (unionByName,allowMissingColumns)- If we get multiple files
#Schema Evolution (orc/parquet with mergeSchema) - If no. of columns are keeps added by the source system
#when we know structure of the file already - schema merge/ schema not known earlier  - schema evolution

#COMBINING OR SCHEMA MERGING or SCHEMA MELTING of Data from different sources(Important interview question also as like schema evolution...)
#4. Multiple files with different structure in multiple paths or sub paths
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)
rawdf_merged=rawdf1.union(rawdf2)#Use union only if the dataframes are having same columns in the same order with same datatype..
display(rawdf_merged)
#Expected right approach to follow
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
display(rawdf_merged)

#Here, we are merging two files because both are in CSV format. If one file is CSV and the other file is in a different format, what should we do in this scenario? it will be handled automatically
#rawdf2.write.json("/Volumes/workspace/wd36schema/ingestion_volume/staging/csvjson")
rawdf3=spark.read.json("/Volumes/workspace/wd36schema/ingestion_volume/staging/csvjson")
rawdf_merged=rawdf_merged.unionByName(rawdf3,allowMissingColumns=True)
display(rawdf_merged)#Expected dataframe to proceed further munging on a single dataframe

**Above multi files handling and schema merging is best handled by DSL rather than SQL.**

####2. Validation, Cleansing, Scrubbing - Cleansing (removal of unwanted datasets), Scrubbing (convert raw to tidy)

In [0]:
#common/generic framework/utilities
def read_file(sparksess,path,format,dell):
    df=sparksess.read.format(format).option("sep",dell).load(path)
    return df
#inline functions

In [0]:
#Validation by doing cleansing
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
#print(rawdf1.schema)
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='permissive')
#cleandf1=read_file(spark,'/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified','csv','|')
print("after keeping nulls on the wrong data format",cleandf1.count())#all rows count
display(cleandf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)
#or
#method2 - drop malformed rows
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='dropMalformed')
print("after cleaning wrong data (type mismatch, column number mismatch)",len(cleandf1.collect()))
display(cleandf1)#We are removing the entire row, where ever data format mismatch is there (throwing away the entire potato)
print(cleandf1.count())#count will return the original count of the raw data
print(len(cleandf1.collect()))#collect+len will return the dropmalformed count of the raw data

#####Validation

In [0]:
#method3 best methodology of applying active data munging
#Validation by doing cleansing (not at the time of creating Dataframe, rather we will clean and scrub subsequently)...
struttype1=StructType([StructField('id', StringType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', StringType(), True), StructField('profession', StringType(), True)])
#method1 - permissive with all rows with respective nulls
rawdf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='permissive')
print("allow all data showing the real values",rawdf1.count())#all rows count
display(rawdf1)#We are making nulls where ever data format mismatch is there (cutting down mud portition from potato)

**Above multi files handling and schema merging is best handled by DSL rather than SQL.**

#####Rejection Strategy

In [0]:
#Creating rejection dataset to send to our source system for future fix
from pyspark.sql.types import StructType,StructField,StringType,ShortType,IntegerType
struttype1=StructType([StructField('id', IntegerType(), True), StructField('firstname', StringType(), True), StructField('lastname', StringType(), True), StructField('age', ShortType(), True), StructField('profession', StringType(), True),StructField("corruptedrows",StringType())])
#method1 - permissive with all rows with respective nulls
cleandf1=spark.read.schema(struttype1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",mode='permissive',columnNameOfCorruptRecord="corruptedrows")
#Create a reject dataset
rejectdf1=cleandf1.where("corruptedrows is not null")
#display(rejectdf1)
rejectdf1.write.csv("/Volumes/workspace/wd36schema/ingestion_volume/staging/reject",mode="overwrite",header=True)
retaineddf1=cleandf1.where("corruptedrows is null")
print("Overall rows in the source data is ",len(cleandf1.collect()))
print("Rejected rows in the source data is ",len(rejectdf1.collect()))
print("Clean rows in the source data is ",len(retaineddf1.collect()))

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cleandf1 (
  id INT,
  firstname STRING,
  lastname STRING,
  age SMALLINT,
  profession STRING,
  corruptedrows STRING
)
USING CSV
OPTIONS (
  path "/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",
  mode "PERMISSIVE",
  columnNameOfCorruptRecord "corruptedrows",
  header "false"
);

In [0]:
display(spark.sql("select * from cleandf1 limit 10"))

display(spark.sql("""
                  SELECT *
FROM cleandf1
WHERE corruptedrows IS NOT NULL limit 10
                  """));

display(spark.sql("""
                  SELECT *
FROM cleandf1
WHERE corruptedrows IS NULL limit 10
                  """));

#####Cleansing 
na.drop()<br>
It is a process of cleaning/removing/deleting unwanted data
Eg. I am purchasing potato from a shop, I am cutting down the debris/rotten portion of it

In [0]:
#We already know how to do cleansing applying the strict Structure on method1 and method2
#Important na functions we can use to do cleansing
cleanseddf=rawdf1.na.drop(how="any")#This function will drop any column in a given row with null otherwise this function returns rows with no null columns - In a scenario of if the source send the Datascience Model features (we shouldn't have any one feature with null value, hence we can use this function)
print("any one row in the raw df with age null")
display(rawdf1.where("age is null"))
print("any one row in the cleansed df with age null")
display(cleanseddf.where("age is null"))#any one column contains null will be cleaned
cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])#If we need CDE without nulls (Critical Data Elements/Significant columns) columns
print("any one row in the cleansed df with id or age null")
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["firstname","lastname"])#4000004,Gretchen,,66,
print("any one row in the cleansed df with firstname and lastname is null")
print("Total rows without firstname and lastname with null values",len(cleanseddf.collect()))
display(cleanseddf)#We are taking this DF further for munging..

In [0]:
%sql
--DSL is better to use, we don't have to manage all the columns in the filter with not() or and operators
--rawdf1.na.drop(how="any",subset=["id","age"])
CREATE OR REPLACE TEMP VIEW cleanseddf_any AS
SELECT *
FROM cleandf1
WHERE id IS NOT NULL AND age IS NOT NULL;

--rawdf1.na.drop(how="all",subset=["firstname","lastname"])
CREATE OR REPLACE TEMP VIEW cleanseddf_all AS
SELECT *
FROM cleandf1
WHERE NOT (firstname IS NULL AND lastname IS NULL);



In [0]:
display(spark.sql("select * from cleanseddf_any limit 10"))
display(spark.sql("select * from cleanseddf_all limit 10"))

#####Scrubbing 
na.fill() & na.replace()<br>
It is a process of polishing/fine tuning/scrubbing/meaningful conversion the data in a usable format
Eg. I am purchasing potato from a shop, I am scrubbing/washing mud/sand portion of it

In [0]:
scrubbeddf1=cleanseddf.na.fill('not provided',subset=["lastname","profession"])#fill will help us replace nulls with some value
display(scrubbeddf1)
find_replace_values_dict1={'Pilot':'Captain','Actor':'Celeberity'}
find_replace_values_dict2={'not provided':'NA'}
scrubbeddf2=scrubbeddf1.na.replace(find_replace_values_dict1,subset=["profession"])#fill function is helping us find and replace the values
scrubbeddf3=scrubbeddf2.na.replace(find_replace_values_dict2,subset=["lastname"])
display(scrubbeddf3)

In [0]:
%sql
--na.fill equivalent cleanseddf.na.fill('not provided',subset=["lastname","profession"])
CREATE OR REPLACE TEMP VIEW scrubbeddf1 AS
SELECT
  id,
  firstname,
  COALESCE(lastname, 'not provided')  AS lastname,
  age,
  COALESCE(profession, 'not provided') AS profession
FROM cleandf1;

--na.replace equivalent scrubbeddf1.na.replace({'Pilot':'Captain','Actor':'Celeberity'})
CREATE OR REPLACE TEMP VIEW scrubbeddf2 AS
SELECT
  id,
  firstname,
  lastname,
  age,
  CASE
    WHEN profession = 'Pilot' THEN 'Captain'
    WHEN profession = 'Actor' THEN 'Celeberity'
    ELSE profession
  END AS profession
FROM scrubbeddf1;


In [0]:
display(spark.sql("select * from scrubbeddf1 limit 10"))
display(spark.sql("select * from scrubbeddf2 limit 10"))

#####DeDuplication
Removal of duplicate rows/columns based on a priority or non priority
distinct & dropDuplicates


In [0]:
display(scrubbeddf3.where("id in ('4000001')"))#before row level dedup
dedupdf1=scrubbeddf3.distinct()#It will remove the row level duplicates
display(dedupdf1.where("id in ('4000001')"))

print("non prioritized deduplication, just remove the duplicates retaining only the first row")
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))
print("prioritized deduplication based on age")
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
#dedupdf1.coalesce(1).where("id in ('4000003')").orderBy(["id","age"],ascending=[True,False]).show(3)
dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))

In [0]:
%sql
--dedupdf2=dedupdf1.coalesce(1).orderBy(["id","age"],ascending=[True,False]).dropDuplicates(subset=["id"])
--better to use qualify
CREATE OR REPLACE TEMP VIEW dedupdf2_priority AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY id
           ORDER BY id,age DESC
         ) AS rn
  FROM scrubbeddf2
) t
WHERE rn = 1;
select * from dedupdf2_priority;

In [0]:
%sql
--show me the duplicate data alone
CREATE OR REPLACE TEMP VIEW duplicateview AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY id ORDER BY id,age DESC) AS rn
  FROM scrubbeddf2
) t;
--select * from duplicateview where rn=1;--deduplicated data
select * from duplicateview where rn>1;--duplicate data alone

####3. Standardization and Replacement / Deletion of Data to make it in a usable format

#####Standardization - 
Making the data more standard by adding/removing/reordering columns as per the expected standard, unifying into expected format, converting the type as expected etc.,

######Standardization1 - Column Enrichment (Addition of columns)

In [0]:
from pyspark.sql.functions import lit,initcap,col
#withColumn("stringcolumnname to add in the df",lit('hardcoded')/initcap(col("colname")))
standarddf1=dedupdf2.withColumn("sourcesystem",lit("Retail"))#SparkSQL - DSL(FBP)
display(standarddf1.limit(20))

In [0]:
%sql
--DSL is better and line of code is less
CREATE OR REPLACE TEMP VIEW standarddf1 AS
SELECT
  *,
  'Retail' AS sourcesystem
FROM dedupdf2_priority 
where upper(id) =lower(id);--wanted to take only number values in id column

select * from standarddf1 limit 10;


######Standardization2 - Column Uniformity

In [0]:
from pyspark.sql.functions import upper
#Basic Exploration/analysis of the profession column for identifying uniformity challenges
#standarddf1.createOrReplaceTempView("sqlview")
#display(spark.sql("select profession,count(*) from sqlview group by profession order by profession"))#SQL
#display(standarddf1.groupBy("profession").count())#DSL
#Standardization2 - column uniformity
standarddf2=standarddf1.withColumn("profession",initcap("profession"))#inicap or any other string function with columnOr name can accept either column or string type provided if the string is a column name for eg. profession/age/sourcesystem.
display(standarddf2.limit(20))
#display(standarddf2.groupBy("profession").count())#DSL

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW standarddf2 AS
SELECT
  id,
  firstname,
  lastname,
  age,
  INITCAP(profession) AS profession,
  sourcesystem
FROM standarddf1;

select * from standarddf2 limit 10;

######Standardization3 - Format Standardization

In [0]:
#Did analysis to understand the format issues in our id and age columns
#standarddf2.where("id like 't%'").show()
standarddf2.where("id rlike '[a-zA-Z]'").show()#rlike is regular expression like function that help us identify any string data in our DF column
standarddf2.where("age rlike '[^0-9]'").show()#checking for any non number values in age column
#standarddf3=standarddf2.withColumn

In [0]:
from pyspark.sql.functions import regexp_replace,replace
#Let's apply scrubbing features to our id column to replace ten with 10 (or we can think of using GenAI here)
replaceval={'one':'1','two':'2','three':'3','four':'4','five':'5','six':'6','seven':'7','eight':'8','nine':'9','ten':'10'}
standarddf3=standarddf2.na.replace(replaceval,["id"])
#standarddf3=standarddf2.withColumn("id",replace("id",lit('ten'),"10"))
standarddf3=standarddf3.withColumn("age",regexp_replace("age",'-',""))
display(standarddf3)

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW standarddf3 AS
SELECT
  CASE
    WHEN cast(id as string) = 'nine'  THEN '9'
    WHEN cast(id as string) = 'ten'   THEN '10'
    ELSE id
  END AS id,
  firstname,
  lastname,
  REGEXP_REPLACE(age, '-', '') AS age,
  profession,
  sourcesystem
FROM standarddf2;

select * from standarddf3 limit 10;


######Standardization4 - Data Type Standardization

In [0]:
standarddf3.printSchema()#Still id and age are string type, though it contains int data
#standarddf4=standarddf3.withColumn("id","id".cast("long"))#this wil not work
standarddf4=standarddf3.withColumn("id",standarddf3.id.cast("long"))
standarddf4=standarddf3.withColumn("id",standarddf3["id"].cast("long"))
standarddf4=standarddf3.withColumn("id",col("id").cast("long"))
standarddf4=standarddf4.withColumn("age",col("age").cast("short"))
standarddf4.printSchema()
display(standarddf4)


In [0]:
%sql
create or replace temp view standarddf4 as 
SELECT 
    CAST(id AS BIGINT) AS id,
    firstname,
  lastname,
  CAST(age AS SMALLINT) AS age,
  profession,
  sourcesystem
FROM standarddf3;

######Standardization5 - Naming Standardization

In [0]:
standarddf5=standarddf4.withColumnRenamed("id","custid")
standarddf5=standarddf4.withColumnsRenamed({"id":"custid","sourcesystem":"srcsystem"})
display(standarddf5)

In [0]:
%sql
create or replace temp view standarddf5 as 
SELECT 
    id custid,
    firstname,
    lastname,
    age,
    profession,
    sourcesystem AS srcsystem
FROM standarddf4;

######Standardization6 - Reorder Standadization

In [0]:
standarddf6=standarddf5.select("custid", "age", "firstname","lastname","profession","srcsystem")
#display(standarddf6)
mungeddf=standarddf6
display(mungeddf.take(10))

In [0]:
%sql
create or replace temp view mungeddf as 
SELECT 
    custid, 
    age, 
    firstname,
    lastname,
    profession,
    srcsystem
FROM standarddf5;

SELECT * FROM mungeddf LIMIT 10;

Before starting Data Enrichment or before sharing the data to the consumer, we have to do EDA/Exploration/Validation

In [0]:
mungeddf.printSchema()
display(mungeddf.take(20))
display("total rows",len(mungeddf.collect()))
display(mungeddf.summary())

##**2. Data Enrichment** - Detailing of data
Makes your data rich and detailed <br>
a. Add (withColumn,select,selectExpr), Derive (withColumn,select,selectExpr), Remove(drop,select,selectExpr), Rename (withColumnRenamed,select,selectExpr), Modify/replace (withColumn,select,selectExpr) - very important spark sql functions <br>
b. split, merge/Concat <br>
c. Type Casting, reformat & Schema Migration <br>

![stage2](stage2.png)

#####a. Add (), Derive (), Rename (), Modify/replace (), Remove/Eliminate () - very important spark sql DF functions

######Adding of columns
Lets add datadt (date of the data orgniated from the source for eg. provided in the filename in a format of yy/dd/MM) and loaddt (date when we are loading the data into our system)

In [0]:
derived_datadt='25/30/12'
print(f"hello '{derived_datadt}'")

In [0]:
from pyspark.sql.functions import lit,current_date#already imported, not needed here
original_filename='custsmodified_25/30/12.csv'#We are deriving this date from the filename provided by the source custsmodified_25/30/12.csv
derived_datadt=original_filename.split('_')[1].split('.')[0]
#derived_datadt='25/30/12'#We are deriving this date from the filename provided by the source custsmodified_25/30/12.csv
enrichdf1=mungeddf.withColumn("datadt",lit('25/30/12')).withColumn("loaddt",current_date())
enrichdf1.printSchema()
#or
enrichdf1=mungeddf.withColumns({"datadt":lit('25/30/12'),"loaddt":current_date()})
enrichdf1.printSchema()
#or
enrichdf1=mungeddf.select("*",lit(derived_datadt).alias('datadt'),current_date().alias('loaddt'))#DSLs (FBP function)
#or
enrichdf1=mungeddf.selectExpr("*","'25/30/12' as datadt","current_date() as loaddt")#DSL(select) + SQL expression
enrichdf1=mungeddf.selectExpr("*",f"'{derived_datadt}' as datadt","current_date() as loaddt")#DSL(select) + SQL expression
enrichdf1.printSchema()
display(enrichdf1)

In [0]:
original_filename='custsmodified_25/30/12.csv'
derived_datadt=original_filename.split('_')[1].split('.')[0]
spark.sql(f"""
    create or replace temp view enrichdf1 as 
    SELECT 
        *,
        '{derived_datadt}' AS datadt,
        current_date() AS loaddt
    FROM mungeddf
""")


In [0]:
%sql
select * from enrichdf1 limit 10;

######Deriving of columns

In [0]:
from pyspark.sql.functions import *
enrichdf2=enrichdf1.withColumn("professionflag",substring("profession",1,1))
#or
enrichdf2=enrichdf1.select("*",substring("profession",1,1).alias("professionflag"))
#or
enrichdf2=enrichdf1.selectExpr("*","substr(profession,1,1) as professionflag")
display(enrichdf2.take(20))

In [0]:
%sql
--Rather than using DSL, a single Select can achieve add/remove/rename/replace/modify the given columns, so sql is better to use for performing all in one select sql
create or replace temp view enrichdf2 as 
SELECT 
    *,
    substring(profession, 1, 1) AS professionflag
FROM enrichdf1;

SELECT * FROM enrichdf2 LIMIT 20;

######Renaming of columns

In [0]:
#Can we use withColumn to rename? not directly, its costly
enrichdf3=enrichdf2.withColumn("sourcename",col("srcsystem"))
enrichdf3=enrichdf3.drop("srcsystem").select("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt","professionflag")
#or
enrichdf3=enrichdf2.select("custid","age","firstname","lastname","profession",col("srcsystem").alias("sourcename"),"datadt","loaddt","professionflag")#costly too, since we have to choose all columns in the select
#or
#enrichdf2.printSchema()
enrichdf3=enrichdf2.selectExpr("custid","age","firstname","lastname","profession","srcsystem as sourcename","datadt","loaddt","professionflag")#costly too, since we have to choose all columns in the select
#or
enrichdf3=enrichdf2.withColumnRenamed("srcsystem","sourcename")#Best function to rename the column(s)
#or
enrichdf3=enrichdf2.withColumnsRenamed({"srcsystem":"sourcename","professionflag":"profflag"})
display(enrichdf3.take(20))

In [0]:
%sql
create or replace temp view enrichdf3 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    profession,
    srcsystem AS sourcename, -- Renaming occurs here
    datadt,
    loaddt,
    professionflag AS profflag -- Renaming occurs here
FROM enrichdf2;

SELECT * FROM enrichdf3 LIMIT 20;

######Modify/replace (withColumn, select/selectExpr)

In [0]:
enrichdf4=enrichdf3.withColumn("profession",col("sourcename"))#This will replace the profession with sourcename
#or
enrichdf4=enrichdf3.withColumn("profession",concat("profession",lit('-'),"profflag"))#This will modify/enrich the profession column with sourcename
#or using select/selectExpr
enrichdf4=enrichdf3.select("custid","age","firstname","lastname",concat("profession",lit('-'),"profflag").alias("profession"),"sourcename","datadt","loaddt","profflag")
#or use selectExpr
enrichdf4=enrichdf3.selectExpr("custid","age","firstname","lastname","concat(profession,'-',profflag) as profession","sourcename","datadt","loaddt","profflag")
display(enrichdf4.take(20))

In [0]:
%sql
create or replace temp view enrichdf4 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    concat(profession, '-', profflag) AS profession, -- Overwrites/Modifies the profession column
    sourcename,
    datadt,
    loaddt,
    profflag
FROM enrichdf3;

SELECT * FROM enrichdf4 LIMIT 20;

######Remove/Eliminate (drop,select,selectExpr) 

In [0]:
#enrichdf4=enrichdf3.withColumn("profession",col("sourcename"))#Cannot be used
#or using select/selectExpr (yes, but costly)
enrichdf5=enrichdf4.select("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt")
#or use selectExpr (yes, but costly)
enrichdf5=enrichdf4.selectExpr("custid","age","firstname","lastname","profession","sourcename","datadt","loaddt")
#or 
enrichdf5=enrichdf4.drop("profflag")#right function to use from dropping
display(enrichdf5.take(20))

In [0]:
%sql
create or replace temp view enrichdf5 as 
SELECT 
    custid,
    age,
    firstname,
    lastname,
    profession,
    sourcename,
    datadt,
    loaddt
FROM enrichdf4;

SELECT * FROM enrichdf5 LIMIT 20;

#####b. Splitting & Merging/Melting of Columns

In [0]:
#Splitting of column
splitdf=enrichdf5.withColumn("profflag",split("profession",'-'))
splitdf=splitdf.withColumn("profession",col("profflag")[0])
#splitdf=splitdf.withColumn("profflag",col("profflag")[1])
#or
splitdf=splitdf.withColumn("shortprof",upper(substring(col("profession"),1,3))).drop("profflag")
#Merging of column
mergeddf=splitdf.select(col("custid"),"age",concat_ws(" ",col("firstname"),col("lastname")).alias("fullname"),"profession","sourcename","datadt","loaddt","shortprof")#usage of select will help us avoid chaining of withColumn,drop,select
display(mergeddf.limit(10))

In [0]:
mergeddf.printSchema()
#unconsious and incompetant(day1)
#consious and incompetant(month3)-current state
#consious and competant(further month)-another few month state
#unconsious and competant(further month)-near end of every stage (pyspark+databricks(sql/python/dwh))

In [0]:
%sql
create or replace temp view mergeddf as 
SELECT 
    custid,
    age,
    concat_ws(' ', firstname, lastname) AS fullname, -- Merging firstname and lastname
    split(profession, '-')[0] AS profession, -- Splitting and taking the first element
    sourcename,
    datadt,
    loaddt,
    upper(substring(split(profession, '-')[0], 1, 3)) AS shortprof -- creating shortprof based on the split result
FROM enrichdf5;

SELECT * FROM mergeddf LIMIT 10;

#####c. Formatting & Typecasting

In [0]:
formatteddf=mergeddf.withColumn("datadt",to_date(col("datadt"),'yy/dd/MM'))#25/30/12 -> 2025-12-30
formatteddf.printSchema()
display(formatteddf.take(10))

In [0]:
%sql
--All the above enrichments can be writtern in a single SQL select.
create or replace temp view formatteddf as 
SELECT 
    custid,
    age,
    fullname,
    profession,
    sourcename,
    to_date(datadt, 'yy/dd/MM') AS datadt, -- Converts string '25/30/12' to Date '2025-12-30'
    loaddt,
    shortprof
FROM mergeddf;
SELECT * FROM formatteddf LIMIT 10;

##3. Data Customization - Application of Tailored Business specific Rules <br>
a. User Defined Functions <br>
b. Building of Frameworks & Reusable Functions (We will learn very next)

![Stage3](stage3.png)

In [0]:
#formatteddf2=formatteddf.withColumn("sourcename",upper("sourcename"))
#formatteddf2.show(2)
#Caveat - If there is no upper() function is available already in spark dsl/sql, we can either search for some functions in the online opensource platform or we have to create one (custom functions)
#from org.apache.sql.functions import upperodd

def upperodd(colname_containsvalue):
    convertedcolvalue=colname_containsvalue.upper()
    return convertedcolvalue
print(upperodd("irfan"))


In [0]:
#If I want to apply a python function on a SQL temp view?
from pyspark.sql.functions import udf
#udfupper=udf(upperodd) this is applicable only for DSL
#Interview question asked: Have you used udf in spark sql? yes, by registering the python function to an UDF function
spark.udf.register("udfuppersql",upperodd) #for SQL udf we have to register the python function to an UDF function
rawdf1.createOrReplaceTempView("rawdf11")
#spark.sql("select custid,udfuppersql(firstname),lastname,age from rawdf11").show(2)

In [0]:
formatteddf2=rawdf1.withColumn("firstname",upper(col("firstname")))#we can't run python function as it is
formatteddf2.explain()
#display(formatteddf2.take(10))#prefer
from pyspark.sql.functions import udf
udfupper=udf(upperodd)#promote normal python function to spark ready udf
formatteddf2=rawdf1.withColumn("firstname",udfupper(col("firstname")))#if udf is inevitable, then we create despite of performance bottleneck
formatteddf2.explain()
display(formatteddf2.take(10))

#####Create Python Custom Function with complex logics

In [0]:
#Calculating age category from the given age of the customer
def pythonAgeCat(dfcol):
    if dfcol is None:
        return "Unknown"
    elif dfcol<=10:
        return "child"
    elif dfcol>10 and dfcol<=18:
        return "teenager"
    elif dfcol>18 and dfcol<=30:
        return "young"
    elif dfcol>30 and dfcol<=50:
        return "middleaged"
    else:
        return "senior"

#####Import udf library, Convert to UDF, Apply in the DF

In [0]:
from pyspark.sql.functions import udf
sparkudfageCat=udf(pythonAgeCat)
customdf=formatteddf.withColumn("agecat",sparkudfageCat("age"))
display(customdf.take(10))

In [0]:
spark.udf.register("sql_age_cat", pythonAgeCat)

In [0]:
%sql
create or replace temp view customdf as 
SELECT 
    *,
    sql_age_cat(age) AS agecat
FROM formatteddf;

SELECT * FROM customdf LIMIT 10;

##4. Data Curation/Processing (Pre Wrangling Stage) - Applying different levels of business logics, transformation, filtering, grouping, aggregation and limits applying different transformation functions
1. Select, Filter
2. Derive flags & Columns
3. Format
4. Group & Aggregate
5. Limit

![Curation](stage4.png)

#####1.Select, Filter
In terms of Performance Optimzation - I ensured to do Push Down Optimization by doing select(project) & Filter(predicate) of what ever the expected data

In [0]:
#Select
#select, functions, case, literal ,from,where,group by, having, order by, limit...
#Select few columns by filtering few rows
selectdf=customdf.select("custid","age","agecat",col("profession").alias("prof"),"agecat")#DSL Select
selectdf.show(5)
selectdf=customdf.selectExpr("custid","age","agecat","profession as prof","agecat")#SQL Select
selectdf.show(5)

In [0]:
#Filter/Where - both are literally same (filter will be used by FBP developers & where will be used by SQL developers)
filterdf=selectdf.filter((col("age")>40) & (col("age")<=60))#DSL operation
filterdf.show(5)
filterdf=selectdf.where((col("age")>40) & (col("age")<=60))#DSL operation
filterdf.show(5)

filterdf=selectdf.filter("age>40 and age<=60")#SQL where operation
filterdf.show(5)
filterdf=selectdf.where("age>40 and age<=60")#SQL where operation
filterdf.show(5)
#filterdf.write.saveAsTable("filtercust")

In [0]:
%python
spark.sql("""select * 
          from customdf
           where age>40 and age<=60;""").createOrReplaceTempView("filterdf")
display(spark.sql("select * from filterdf"))


In [0]:
%sql
--sql is familiar to use rather than using dsl
create or replace temp view filterdf as 
select * from customdf
where age>40 and age<=60;
select * from filterdf;

#####2. Derive flags & Columns

In [0]:
#We have created agecat using UDF (which is supposed to use only if it is inevitable)
#But we can do the same using DSL When.otherwise or SQL CASE WHEN
#Deriving Flag
#Syntax in DSL: when(conditions,"value").when(conditions,"value2").otherwise("valuen").alis("colname")
curateddf=customdf.select("*",when(col("age").isNull(),"U").
                          when(col("age")<=10,"C").
                          when((col("age")>10) & (col("age")<=18),"T").
                          when((col("age")>18) & (col("age")<=30),"Y").
                          when((col("age")>30) & (col("age")<=50),"M").
                          otherwise("S").alias("agecatflag"))#Suggessted than using UDFs
display(curateddf.take(10))
#Deriving Column
#Syntax in SQL: case when conditions then value when conditions then value2 else valuen end as colname
curateddf=curateddf.drop("agecat").selectExpr(
    "*","""CASE when age is NULL then 'Unknown' 
                              when age<=10 then 'child' 
                              when age>10 and age<=19 then 'teenager' 
                              when age>19 and age<=30 then 'young'
                              when age>30 and age<=50 then 'middleaged'
                              else 'oldaged' end as agecat""")#Suggessted than using UDFs
display(curateddf.take(10))
#Interview Answer of how you optimized the existing spark code developed by your ex team members?
#I analysed the existing udfs used in my project and seeked for opurtunities to convert them into SQL/dsl based programs by implementing the udf logics.

In [0]:
%sql
--SQL is better to use than DSL because SQL familiar and simplified syntax
--One drawback of DSL is, DSL is platform agnostic (work only in Spark DF) but SQL is non-platform agnostic (works anywhere)
create or replace temp view curateddf as 
select *,case when age is null then 'Unknown' 
                              when age<=10 then 'child' 
                              when age>10 and age<=19 then 'teenager' 
                              when age>19 and age<=30 then 'young'
                              when age>30 and age<=50 then 'middleaged'
                              else 'oldaged' end as agecat1 
                              from customdf ;
select * from curateddf;

In [0]:
'''def pythonAgeCat(dfcol):
    if dfcol is None:
        return "Unknown"
    elif dfcol<=10:
        return "child"
    elif dfcol>10 and dfcol<=18:
        return "teenager"
    elif dfcol>18 and dfcol<=30:
        return "young"
    elif dfcol>30 and dfcol<=50:
        return "middleaged"
    else:
        return "senior"'''

#####3.Format (Deriving Columns with different format)

In [0]:
#We can use different functions - string or number or date function for format modeling
curateddf3=curateddf.select("*",datediff("loaddt","datadt").alias("delaydays"),year("datadt").alias("datayear")
                            ,month("datadt").alias("datamonth")
                            ,last_day("datadt").alias("datalastday")).withColumn("agecat",initcap("agecat"))
display(curateddf3.take(3))

In [0]:
%sql
create or replace temp view curateddf3 as 
select *,
datediff(loaddt,datadt) as delaydays,year(datadt) as datayear
                            ,month(datadt) as datamonth
                            ,last_day(datadt)as datalastday,initcap(agecat) as agecat2 from curateddf;
select * from curateddf3;

#####4. Group & Aggregate
Before performing grouping or aggr, consider the below factors from the dataset....<br>
identifier?	cid (high in cardinality/difference) (surrogate/naturalkey)<br>
descriptive?	name<br>
metric?	avg(age),count(distinct cid),max(age),min(age)<br>
measure?	age,cid<br>
grouping?	age,prof - low in cardinality/difference<br>

In [0]:
#from pyspark.sql.functions.aggregate import avg,count,initcap,last_day,datediff,year,month,agg
#What is the total number of customers we have?
print(curateddf3.count())
#What is the total number of customers we have in each profession?
curateddf4=curateddf3.groupBy("profession").count()
display(curateddf4.take(100))
#Multiple Aggregation with one grouping - What is the total number of customers,average age of those customers we have in each profession?
curateddf4=curateddf3.groupBy("profession").avg("age").withColumnRenamed("avg(age)","avgage")
display(curateddf4.take(100))
#To calculate multiple aggregation, we need to use a function called agg function
curateddf4=curateddf3.groupBy("profession").agg(count("custid").alias("custcount"),avg("age").alias("avgage"))
display(curateddf4.take(100))
#curateddf4 this dataframe we materialize/store in some tables/files later
#Multiple Aggregation with multiple grouping - What is the total number of customers,average age of those customers we have in each profession?
curateddf4=curateddf3.groupBy("profession","agecat").\
agg(count("custid").alias("custcount"),avg("age").alias("avgage"))
display(curateddf4.take(100))



In [0]:
%sql
--SQL is more familiar and easy for lift and shift operation
create or replace temp view curateddf4 as 
select profession,agecat,count(custid) as custcount,avg(age) as avgage
from curateddf3
group by profession,agecat;
select * from curateddf4 limit 10;

#####5. Ordering

In [0]:
curateddf5=curateddf4.orderBy("profession","agecat")
display(curateddf5.take(100))

In [0]:
%sql
create or replace temp view curateddf5 as 
select *
from curateddf4
order by profession,agecat;
select * from curateddf5 limit 10;

#####6. Limit
Let us take an oppurtunity to understand different data limiting/restricting functions<br>
Limit is a dataframe TRANSFORMATION functions used to limit the number of rows returned in a spark dataframe FORMAT<br>
Take is a dataframe/RDD ACTION functions used to limit the number of rows returned in a python list FORMAT<br>
display is a standard output databricks specific function used to produce entire DF/List output in a notebook view with multiple options <br>
show is a standard output spark dataframe specific function used to produce default 20 rows of a DF output in a notebook/REPL/IDE view <br>
collect is a spark action that help us collect the DF/RDD data into the driver environment in a form of python list <br>

In [0]:
#anything can be used under display()
print("limit output")
curateddf5.limit(20).show(10)
print("take output")
curateddf5.take(10)
#When to use what
#I have to filter some data in a limited dataset of 100 rows
curateddf5.limit(100).filter("profession='Accountant'").show()
#Display
display(curateddf5.limit(100).filter("profession='Accountant'"))

##5. Data Wrangling - More of Analytics + Transformation
  1. Joins - Relation/Connection established between one or more datasets/df/tabl to produce the broader/extended view of the data horizontally.
  5 Categories of Joins:
  inner, outer(left
  right, full), self, cross, special optimized (semi, anti)
  2. Lookup
  3. Lookup & Enrichment
  4. Schema Modeling  (Denormalization)
  5. Windowing
  6. Analytical
  7. Set operations
  8. grouping & aggregations

![Stage5](stage5.png)

#####1. Joins
Joins are Relation/connection of one or more tables to perform widened (horizontal) data analytics
1. Frequently used simple joins (inner, left)
2. InFrequent simple joins (self, right, full, cartesian)
3. Advanced joins (Semi and Anti)<br>
Syntax - dfleft.join(dfright,how='typeofjoin',on='custid'=='cid')
4. Optimized joins (Broadcast join, SMB Join, Shuffle hash join, Map/Reduceside join, Skewed join etc.)-AQE (Adaptive Query Execution)

In [0]:
#Ques1: In what order i can apply the functions in dataframe curation/transformation
rawdf1=rawdf1.na.drop().where("id<>'ten' and id<>'trailer_data:end of file'")
rawdf1.explain()
rawdf1=rawdf1.where("id<>'ten' and id<>'trailer_data:end of file'").na.drop()
rawdf1.explain()
#Ques2:If I wanted to join more than 1 tables
rawdf2=rawdf1.withColumn("multiconcat",concat(concat(lit('a'),lit('b'))))
rawdf3=rawdf2
joineddf1=rawdf1.join(rawdf2,how='inner')
rawdf4=rawdf2
joineddf1.join(rawdf3,how='left').join(rawdf4,how='right')


In [0]:
from pyspark.sql.functions import lit,col
#How to write join syntax in Spark and learn the semantics of join in spark
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",header=False,inferSchema=True).toDF("id","firstname","lastname","age","profession")
rawdf1=rawdf1.na.drop().where("id<>'ten' and id<>'trailer_data:end of file'")
rawdf1=rawdf1.where("id<>'ten' and id<>'trailer_data:end of file'").na.drop()
leftdf=rawdf1.select("id","age","firstname","lastname").where("id in (4000100,4000101)")
rightdf=rawdf1.select("id","profession").where("id in (4000100,4000102,4000103)")
leftdf.show(20,False)
rightdf.show(20,False)
#Let's understand all types of joins syntax & semantics quickly
#dfleft.join(dfright,how='typeofjoin',on='custid'=='cid')

#5 Categories of Joins : 1. inner(important), 2. outer(left(important), right, full), 3. special optimized(important) (semi, anti), 4. self, 5. cross
#1. inner, 2. outer(left, right, full)
#1. inner
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("inner")
innerjoindf=leftdf.join(rightdf,how='inner',on='id')
innerjoindf.show(20)#only 4000100 from both df is displayed

#2. outer(left, right, full)
#2. left
#Usecase - Ensure all of the master customers with/without profession provided - Functionality(application)
print("left")
leftjoindf=leftdf.join(rightdf,how='left',on='id')#syntax(how to do)
leftjoindf.show(20)#semantics(what is the output) - only 4000100,4000101 from both df is displayed with non applicable nulls in the right

#2. right
#Usecase - Ensure all of the customers with professions has to be displayed
print("left")
rightjoindf=leftdf.join(rightdf,how='right',on='id')#syntax(how to do)
rightjoindf.show(20)#semantics(what is the output) - only 4000100,4000102,4000103 from both df is displayed with non applicable nulls in the left

#2. full
#Usecase - Ensure all of the customers with/without master customers or professions has to be displayed
print("full")
fulljoindf=leftdf.join(rightdf,how='full',on='id')#syntax(how to do)
fulljoindf.show(20)#semantics(what is the output) - displays all 4000100,4000101,4000102,4000103 from both df is displayed with non applicable nulls from both left and right

#3. special optimized (semi, anti)
#Why semi/anti is an optimized join?
#Interview answer - I found low hanging fruits oppurtunities in my project.. converting inner/left joins to semi/anti, because the joins are good in performance (it uses exists condition rather than in condition)
#semi
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("left semi/semi")
semijoindf=leftdf.join(rightdf,how='leftsemi',on='id')
semijoindf.show(20)#only 4000100 (matches between both df) from LEFT df is displayed

#anti
#Usecase - Ensure which ever the customers (from both df) matches - Functionality(application)
print("left anti/anti")
antijoindf=leftdf.join(rightdf,how='leftanti',on='id')
antijoindf.show(20)#only 4000101 (un matched between both df) from LEFT df is displayed

#4. Self join
#Usecase - Hierarchical Retrival or join - Data joined by itself to produce the relational output of the self dataset
print("self")
custaffliatedf=leftdf.withColumn("refcustid",lit('4000100'))
custaffliatedf.show(3)
selfjoindf=custaffliatedf.alias("l").join(custaffliatedf.alias("r"),how='inner',on=(col('l.id')==col("r.refcustid")))

selfjoindf.select('l.id','r.*').show(20)#only 4000101 (un matched between both df) from LEFT df is displayed

#5. cross join
#What join it will perform by default? inner join (if we use on condition), cartesian/cross join (if no on condition used)
#Minimum syntax to use?
#cross
print('cartesian/cross')
joindf=leftdf.join(rightdf)#without on it is cross/cartesian product (very costly and avoidable join)
joindf.show(20)#6 rows returned

print('inner by default')
joindf=leftdf.join(rightdf,on='id')#default inner join
joindf.show(20)#4000100 returned

print('expecting right join, but because of lack of on, cross join happened')
joindf=leftdf.join(rightdf,how='right')#without on it is cross/cartesian product (very costly and avoidable join)
joindf.show(20)#6 rows returned



#####Applications of Joins
Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last year, so i can send them some offers.

In [0]:
from pyspark.sql.types import *
#We already have customer master (Qualifying/dimension) data in a curated state
curateddf3.show(5)#This contains only customer curated data (munged, enriched, customized, curated)
#We need to bring transaction detailed (Quantifying/Fact) data upto curated state (we need to follow all/any/none of the steps that we followed so far munged, enriched, customized, curated, wrangle etc., depends on the EDA result)
#strt1="txnid long,txndt string,custid int,amt double,category string,product string,city string,state string,spendby string"
strt1=StructType([StructField('txnid', IntegerType(), True), StructField('txndt', StringType(), True), StructField('custid', IntegerType(), True), StructField('amt', DoubleType(), True), StructField('category', StringType(), True), StructField('product', StringType(), True), StructField('city', StringType(), True), StructField('state', StringType(), True), StructField('spendby', StringType(), True)])
#txnsrawdf=spark.read.csv("/Volumes/workspace/default/volumewd36/txns_2025.txt",inferSchema=True,header=False)
#txnsrawdf.schema
txnsrawdf=spark.read.schema(strt1).csv("/Volumes/workspace/default/volumewd36/txns_2025.txt",header=False)
txnsrawdf.printSchema()
txnsrawdf.show(5)
txnsrawdf.summary().show(100)
#As per the above output, we can do some transformations of munging, enrichment etc.,
txnsmungeddf=txnsrawdf.na.drop().dropDuplicates(["txnid"])
txnsenrichdf=txnsmungeddf.withColumn("surrogatekey",monotonically_increasing_id())
txnscurateddf=txnsenrichdf.withColumn("txndt",to_date(col("txndt"),"MM-dd-yyyy"))
txnscurateddf.printSchema()
txnscurateddf.show(5)#Completed munging, enrichment and curation

#Now lets achieve the solution for the business requirement:
#Requirement: I need to do analytics/reporting of top 3 customers who did highest amount of transactions in our business last month, so i can send them some offers.
#spark.sql("select month(add_months(current_date(), -1))").show()
txnsdecdf=txnscurateddf.where("month(txndt)=month(add_months(current_date(), -1))")
print("dec transactions count ",txnsdecdf.count())

custdimdf=curateddf3
joineddf=custdimdf.join(txnsdecdf,how="inner",on="custid")
joineddf.show(10)
#We have to generate a reporting table with only custid,name,profession,amount,spendby
reportdf=joineddf.select("custid","fullname","amt","spendby").orderBy(["amt"],ascending=[False]).limit(3)
reportdf.show()#Top 3 transacting customer info (in the overall data)
reportdf=joineddf.select("custid","fullname","amt","spendby").orderBy(["amt"],ascending=[True]).limit(3)
reportdf.show()#Least 3 transacting customer info (in the overall data)


######1. Lookup (only exists/non exists check) - semi or anti
Lookup is the process of looking up for some data attributes using the key to identify the presence of the values (not the actual values are returned)
Eg. whether this particular customer made a transaction or not


In [0]:
#Lookup: Show me only the customer information, who did transactions last month (i dont need what transaction)?
#What type of best join i have to use? semi join is best and faster
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='semi',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())
#I store this data in a table, my business analysts will apply filter a particulatr customer

#It is possible to produce the same result by using inner or left join also, but not preferred just for lookup
custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='inner',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).distinct()
print(resultdf.count())

custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='left',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).where("txnid is not null").distinct()
print(resultdf.count())

#Lookup: Show me only the customer information, who did not do transactions last month?
custonlylastmonthnontransacteddf=custdimdf.join(txnsdecdf,how='anti',on='custid')
custonlylastmonthnontransacteddf.show(2)
print(custonlylastmonthnontransacteddf.count())

custonlylastmonthtransacteddf=custdimdf.alias("cust").join(txnsdecdf,how='left',on='custid')
resultdf=custonlylastmonthtransacteddf.select(custdimdf["*"]).where("txnid is null").distinct()
print(resultdf.count())

In [0]:
custdimdf.createOrReplaceTempView("custdimdftv")
txnsdecdf.createOrReplaceTempView("txnsdecdftv")

In [0]:
%sql
--Wanted to do lookup, whether customer has transacted in last month or not?
create or replace temp view custonlylastmonthtransacteddf as 
SELECT cust.* 
--cust.age,cust.fullname,cust.profession,cust.sourcename,cust.datadt, cust.loaddt,cust.shortprof,cust.agecatflag,cust.agecat,t.*
FROM custdimdftv cust
LEFT SEMI JOIN txnsdecdftv t ON cust.custid = t.custid;
select * from custonlylastmonthtransacteddf limit 10;

######2. Lookup & Enrichment
Lookup and enrichment using joins - left_join (best), inner, right/full(least bother)
Lookup and enrichment is the process of looking up for for some data attributes using the key and enrich the values

In [0]:
#Lookup & Enrichment: Show me both the customer information and transactions information of last month (i need what are the transactions made by the given customer)?
#What type of best join i have to use? inner (only those who did transactions) or left join (all customers)
#left
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='left',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())
#inner
custonlylastmonthtransacteddf=custdimdf.join(txnsdecdf,how='inner',on='custid')
custonlylastmonthtransacteddf.show(2)
print(custonlylastmonthtransacteddf.count())

#We can't achieve the same result by using semi join, because it will not return the transaction info

In [0]:
%sql
--Wanted to do lookup & enrichment, transactions details are enriched for the given customer
create or replace temp view custonlylastmonthtransacteddf as 
SELECT cust.age,cust.fullname,cust.profession,cust.sourcename,cust.datadt, cust.loaddt,cust.shortprof,cust.agecatflag,cust.agecat,t.*
FROM custdimdftv cust
LEFT JOIN txnsdecdftv t ON cust.custid = t.custid;
select * from custonlylastmonthtransacteddf limit 10;

######3.Schema Modeling (Denormalization-joined result of tables)-DWH/DataMart<br>
Use inner(mostly),left/right/full(we can use depends on the business)
We can build Datawarehouse components (dimension, fact tables) appling joins on the tables to achieve different types of schemas
1. Star Schema
txns_fact cust_mast_dim(dim)    
id,amt    cid,name,age,city_lived 
1,100     11,irfan,44,Bangalore|Chennai
2. Snowflake Schema - 
txns_fact cust_mast_dim(dim)    cust_det_dim(subdim)
id,amt    cid,name,age          cid,city_lived
1,100     11,irfan,44           11,Bangalore
                                11,Chennai

In [0]:
#Implementing Star schema model
#In our current dataset, we have what schema model can be defined? Only star schema is possible, because we have only one fact table and multiple dimension tables (no sub dimension).
#What is the best way to join all the tables? We can use inner join, because we have only one fact table and multiple dimension tables.
#inner
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='inner',on='custid')
denormalized_fat_wide_df.show(2)
#I store this denormalized dataframe data into final fact table (in a single table) to simplify my customer queries without applying joins.
print(denormalized_fat_wide_df.count())

#or We can use left join also to ensure all customer info is captured.
#left
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='left',on='custid')
denormalized_fat_wide_df.show(2)
print(denormalized_fat_wide_df.count())

#or We can use full join also to ensure all customer (and+or) transaction info is captured.
#left
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='full',on='custid')
denormalized_fat_wide_df.show(2)
print(denormalized_fat_wide_df.count())

#####4.Windowing Functionalities
Is the concept of grouping/bucketize/dividing/partitioning and performing some analytical operation on the literally partitioned data
Benifits of Windowing Functionality
1. Creating Surrogate/primary key/seq number
2. Performing Top N Analysis
3. Duplicate Handling

In [0]:
from pyspark.sql.window import Window
denormalized_fat_wide_df=custdimdf.join(txnsdecdf,how='inner',on='custid')
print(denormalized_fat_wide_df.count())
orderjoineddf=denormalized_fat_wide_df.where("custid in (4000022,4000816)").select("custid","age","profession","txndt","amt","category","product","city","state","spendby")
#display(orderjoineddf)

#We are using row_number() window function (very important)
#Synax for windowing function
#from pyspark.sql.window import Window
#select(row_number().over(Window.partitionBy("custid").orderBy("txndt"))).alias("seqnum"))

#1.Creating Surrogate/primary key/seq number
#Let us perform a non windowing operation that means we are not using partitioning
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.orderBy("custid","txndt")))#overall sorting
display(sk_orderjoinedf)

#2. Performing Top N Analysis
#Let us perform windowing operation
#Interview Questions pattern:
print("a. Tell me the top 1 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum=1")
display(sk_orderjoinedf)
print("b. Tell me the least 1 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("amt"))).where("seqnum=1")
display(sk_orderjoinedf)
print("c. Tell me the top 2 transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum<=2")
display(sk_orderjoinedf)
print("d. Tell me the just the 2nd highest transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy(desc("amt")))).where("seqnum=2")
display(sk_orderjoinedf)
print("e. Tell me the just the 2nd highest transaction made by the customer within the given state")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid","state").orderBy(desc("amt")))).where("seqnum=2")
display(sk_orderjoinedf)

print("f. Show me the very first time the business made by the given customers?")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("txndt"))).where("seqnum=1")
display(sk_orderjoinedf)
print("g. Show me the very first two transaction made by the given customers?")
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid").orderBy("txndt")))\
.where("seqnum<=2")
display(sk_orderjoinedf)

#3. Duplicate Handling
#I want to remove duplicates based on custid with same spendby (for a given customer i need only one credit and one cash transaction info)
print("I want to get the duplicate custid with the same spendby removed out from our data?")
#display(sk_orderjoinedf.coalesce(1).dropDuplicates(["custid","spendby"]))
#Using windowing function we can do controlled way of dropping duplicates, for eg. i want to drop the repeating data of same customer and spendby by retaining only the latest transactions
sk_orderjoinedf=orderjoineddf.withColumn("seqnum",row_number().over(Window.partitionBy("custid","spendby").orderBy(desc("txndt"))))
display(sk_orderjoinedf)
display(sk_orderjoinedf.where("seqnum=1"))

#but if amt is same for 2 transaction of diff dates? - rank

In [0]:
#Second important window functions are rank() - used for applying same rank for same values and will have gaps (don't maintain continuity) 
# and dense_rank() - used for applying same rank for same values and will not have gaps (maintain denser/close continuity) 
print("a. Tell me the just the 2nd highest transaction made by the customer")
sk_orderjoinedf=orderjoineddf.withColumn("rnk",rank().over(Window.partitionBy("custid").orderBy(desc("spendby")))).withColumn("densernk",dense_rank().over(Window.partitionBy("custid").orderBy(desc("spendby"))))
display(sk_orderjoinedf)
orderjoineddf.createOrReplaceTempView("orderjoineddftv")

In [0]:
%sql
--Comparing with DSL, SQL is more familiar and simple in syntax for writing and reasoning.
--orderjoineddf.withColumn("rnk",row_number().over(Window.partitionBy("custid").orderBy(desc("spendby")))).withColumn("densernk",dense_rank().over(Window.partitionBy("custid").orderBy(desc("spendby"))))
--no need of extending pyspark.sql.window import Window, no need of desc(), no need of using camelCase functions etc.,
select *,
row_number() over(partition by custid order by txndt desc) as rno,
rank() over(partition by custid order by txndt desc) as rnk,
dense_rank() over(partition by custid order by txndt desc) as drnk
from orderjoineddftv 
limit 100;

#####5.Analytical Functionalities
Performing analytics/summarization/categorization of data by applying/not by applying windowing functionality.

In [0]:
#Comparitive analytics - lead & lag
sk_orderjoinedf=orderjoineddf.withColumn("nexttransamt",lead("amt",1,0).over(Window.partitionBy("custid").orderBy(asc("txndt")))).withColumn("priortransamt",lag("amt",1,0).over(Window.partitionBy("custid").orderBy(asc("txndt"))))
aggranalyticaldf=sk_orderjoinedf
display(sk_orderjoinedf)

In [0]:
decoded_df=sk_orderjoinedf.withColumn("transpattern",when(col("priortransamt")==lit(-1),"first transaction").when(col("amt")>col("priortransamt"),"increase").when(col("amt")<col("priortransamt"),"decrease").otherwise("same"))
decoded_df.show(2)

In [0]:
%sql
create or replace temp view aggranalytical_tv as
SELECT 
    *,
    CASE 
        WHEN priortransamt = -1 THEN 'first transaction'
        WHEN amt > priortransamt THEN 'increase'
        WHEN amt < priortransamt THEN 'decrease'
        ELSE 'same'
    END AS transpattern
FROM
    (SELECT 
        *,
        LAG(amt, 1, -1) OVER (PARTITION BY custid ORDER BY txndt ASC) AS priortransamt,
        LEAD(amt, 1, -1) OVER (PARTITION BY custid ORDER BY txndt ASC) AS nexttransamt
    FROM orderjoineddftv ) lag_lead_calculated;

select * from aggranalytical_tv limit 100;

In [0]:
sk_orderjoinedf=orderjoineddf.withColumn("firsttransvalue",first_value("amt").over(Window.partitionBy("custid").orderBy(asc("txndt")))).withColumn("lasttransvalue",first_value("amt").over(Window.partitionBy("custid").orderBy(desc("txndt")))).withColumn("cumedistribution",cume_dist().over(Window.partitionBy("custid").orderBy(desc("txndt"))))
display(sk_orderjoinedf)

In [0]:
%sql
SELECT 
    *,
    FIRST_VALUE(amt) OVER (
        PARTITION BY custid 
        ORDER BY txndt ASC
    ) AS firsttransvalue,
    FIRST_VALUE(amt) OVER (
        PARTITION BY custid 
        ORDER BY txndt DESC
    ) AS lasttransvalue,
    CUME_DIST() OVER (
        PARTITION BY custid 
        ORDER BY txndt DESC
    ) AS cumedistribution
FROM orderjoineddftv;

In [0]:
#Identify the identifier(custid), dimensions(age,profession,txndt,category,product,city,state,spendby), measures(amt) and what metrics (sum,avg,mean,mode,variance...) we needed based on what dimensions (product,spendby).
#Let's calculate the product wise sum(amt) metrics
groupaggdf=orderjoineddf.groupBy("product").agg(sum("amt").alias("sumamt"))
display(groupaggdf)
#Let's calculate the product wise sum(amt) metrics
groupaggdf=orderjoineddf.where("product<>'Exercise & Fitness'").groupBy("product","spendby").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).where("sumamt>100").orderBy(["product","sumamt"],ascending=[True,False])
display(groupaggdf)

In [0]:
%sql
--sql is more familiar and english like language
SELECT 
    product,
    spendby,
    SUM(amt) AS sumamt,
    AVG(amt) AS avgamt
FROM orderjoineddftv
where product<>'Exercise & Fitness'
GROUP BY product, spendby
having SUM(amt)>100
ORDER BY 
    product ASC,
    sumamt DESC;

In [0]:
#rollup - rolled up aggregation at state level and no level
rollupdf=orderjoineddf.rollup("state","city").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).orderBy("state","sumamt")
display(rollupdf)

#cube - cubed (all level) aggregation at state level, city level and no level
cubedf=orderjoineddf.cube("state","category").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).orderBy("state","sumamt")
display(cubedf)

#cube - cubed (all level) aggregation at state level, city level and no level
cubedf=orderjoineddf.cube("state","category").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).orderBy("state","sumamt")
display(cubedf)

#pivot
groupdf=orderjoineddf.groupBy("state","city","category").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).orderBy("state")
display(groupdf)
pivotdf=orderjoineddf.groupBy("state","city").pivot("category").agg(sum("amt").alias("sumamt"),avg("amt").alias("avgamt")).orderBy("state")
display(pivotdf)

In [0]:
%sql
SELECT
    state,
    city,
    SUM(amt) AS sumamt,
    AVG(amt) AS avgamt
FROM orderjoineddftv
GROUP BY ROLLUP (state, city)
ORDER BY state, sumamt;

In [0]:
%sql
SELECT
    state,
    category,
    SUM(amt) AS sumamt,
    AVG(amt) AS avgamt
FROM orderjoineddftv
GROUP BY CUBE (state, category)
ORDER BY state, sumamt;

In [0]:
%sql
--Pivot in SQL is not dynamic/not simple as like DSL, hence DSL is better option for doing Pivot
SELECT * FROM (
  SELECT 
    state, 
    city, 
    category, 
    amt 
  FROM orderjoineddftv
) as originaldata
PIVOT (
  SUM(amt) AS sum_amt, 
  AVG(amt) AS avg_amt
  FOR category IN ('Exercise & Fitness', 'Jumping')--Additional effort in SQL
)
ORDER BY state;

In [0]:
%sql
SELECT 
    state,
    city,
    SUM(CASE WHEN category = 'Exercise & Fitness' THEN amt ELSE 0 END) as exercise_fitness_Sum,
    SUM(CASE WHEN category = 'Jumping' THEN amt ELSE 0 END) as Jumping_Sum
FROM orderjoineddftv
GROUP BY state, city
ORDER BY state;

#####6.Set operations

In [0]:
from pyspark.sql.functions import lit,col
rawdf1=spark.read.csv("/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified",header=False,inferSchema=True).toDF("custid","firstname","lastname","age","profession")
cleandf1=rawdf1.where("custid not in ('ten','trailer_data:end of file')")

In [0]:
cleandf1=rawdf1.where("custid not in ('ten','trailer_data:end of file')")
df1=cleandf1.where("custid between 4000100 and 4000110")
df2=cleandf1.where("custid between 4000105 and 4000115")
df1.show()
df2.show()
#set operations
#Thumb rules to perform set operations:
#1. Column numbers should be same
#2. Column order should be same
#3. Column datatype should be same

#union (returns union of multiple dfs with duplicates)
uniondf=df1.union(df2)#returns duplicates
uniondf=df1.union(df2).distinct()#returns distinct result
print("union ",uniondf.count())
uniondf.show(100)

#unionall (alias of union/same as union)
unionalldf=df1.unionAll(df2)
print("union All",uniondf.count())
unionalldf.show(100)

#unionByName
df2=df2.select("custid","age","profession","firstname","lastname",lit("retailsource").alias("sourcesystem"))
unionbyname=df1.unionByName(df2,allowMissingColumns=True)
unionbyname.show(100)

#intersect (returns common data between both df excluding duplicates)
df2=cleandf1.where("custid between 4000105 and 4000115")
intersectdf=df1.intersect(df2)
print("intersection",intersectdf.count())
intersectdf.show(100)

#intersectall (returns common data between both df including duplicates)
df2=cleandf1.where("custid between 4000105 and 4000115")
intersectdf=df1.intersectAll(df2)
print("intersection",intersectdf.count())
intersectdf.show(100)

#subtract (returns df1 - df2)
df2=cleandf1.where("custid between 4000105 and 4000115")
subtractdf=df1.subtract(df2)
print("df1 subtraction df2",subtractdf.count())
subtractdf.show(100)


In [0]:
%sql
CREATE OR REPLACE TEMP VIEW df1 AS
SELECT *
FROM cleandf1
WHERE id BETWEEN 4000100 AND 4000110;

CREATE OR REPLACE TEMP VIEW df2 AS
SELECT *
FROM cleandf1
WHERE id BETWEEN 4000105 AND 4000115;

--set operations
--Thumb rules to perform set operations:
--1. Column numbers should be same
--2. Column order should be same
--3. Column datatype should be same


In [0]:
%sql
--union (returns union of multiple dfs with duplicates)
SELECT * FROM df1
UNION
SELECT * FROM df2;


In [0]:

%sql
--unionall (alias of union/same as union)
SELECT * FROM df1
UNION ALL
SELECT * FROM df2;



In [0]:
%sql
--unionByName (we don't have available in SQL, we need to handle explicitly, hence DSL is better for schema merging)
SELECT
    id,age, profession,firstname,lastname,'retailsource' sourcesystem
FROM df1
UNION ALL
SELECT id,age,profession,firstname,lastname,'retailsource' AS sourcesystem
FROM df2;




In [0]:
%sql
--intersect (returns common data between both df excluding duplicates)
SELECT * FROM df1
INTERSECT
SELECT * FROM df2;




In [0]:
%sql
--subtract (returns df1 - df2)
SELECT * FROM df1
minus
SELECT * FROM df2;

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>
Enablement of the Cleansed, transformed and analysed data as a Data Product.<br>
Consumer Data Product

In [0]:
#Target/Publishing layers are either Datalake(DBFS(S3)) or Lakehouse(Deltalake tables)
mungeddf.write.json("/Volumes/workspace/wd36schema/ingestion_volume/destination/mungedjson",mode='overwrite')
curateddf4.write.format("delta").save("/Volumes/workspace/wd36schema/ingestion_volume/destination/curateddelta",mode='overwrite')
aggranalyticaldf.write.saveAsTable("workspace.wd36schema.aggranalyticaldf_table",mode='overwrite')
#Wanted to understand how to store/persist tempview into file and table?
#transfer this tempview into to df and write into table/file
#how to convert a tv to a df? using spark.sql function
spark.sql("select * from aggranalytical_tv").write.format("delta").save("/Volumes/workspace/wd36schema/ingestion_volume/destination/curateddeltatv",mode='overwrite')
#We dont need to transfer this tempview into to df to write into table, rather we can directly write TV into Table?
spark.sql("create or replace table workspace.wd36schema.aggranalyticaltv_table as select * from aggranalytical_tv")